### Notebook to compile the facs data table from the FlowJo output (+IndexSort plugin).
.fcs files were gated in FlowJo and well location was preserved using the IndexSort plugin. Bi-exponential transform was applied and the FACS data was exported as the transformed 'channel' tables. To preserve the well location, also the un-transformed 'scale' tables were exported. These tables are beeing merged in this notebook.

In [1]:
import pandas as pd
import os

In [2]:
# 300ms
plates = ['20759']
path = '../data/300ms/FJ/'
# 500ms
plates = ['20761']
path = '../data/500ms/FJ/'
# bulk_b
plates = ['Specimen_001_INX_Bulk_002_004',
          'Specimen_001_INX_Bulk_003_005',
          'Specimen_001_INX_Bulk_006_008']
path = '../data/bulk_b/FJ/'
# bulk_c
plates = ['Specimen_001_INX_Bulk_001_003',
          'Specimen_001_INX_Bulk_004_006',
          'Specimen_001_INX_Bulk_005_007']
path = '../data/bulk_c/FJ/'
# bulk_b_empty
plates = ['8227_INX_Bulk_P1_002',
          '8227_INX_Bulk_P2_003',
          '8227_INX_Bulk_P3_004']
path = '../data/bulk_b_empty/FJ/'
# enrich
plates = ['Specimen_001_INX_Celltype_P2_100920_004']
path = '../data/enrich/FJ/'



plates = ['20759', # 300ms
          '20761', # 500ms
          'Specimen_001_INX_Bulk_002_004', # bulk_b_1
          'Specimen_001_INX_Bulk_003_005', # bulk_b_2
          'Specimen_001_INX_Bulk_006_008', # bulk_b_3
          'Specimen_001_INX_Bulk_001_003', # bulk_c_1
          'Specimen_001_INX_Bulk_004_006', # bulk_c_1
          'Specimen_001_INX_Bulk_005_007', # bulk_c_1
          '8227_INX_Bulk_P1_002', # bulk_b_e_1
          '8227_INX_Bulk_P2_003', # bulk_b_e_2
          'Specimen_001_INX_Celltype_P2_100920_004'] # enrich

path = '../data/facs_data/'

In [3]:
# get all csv files in channel and scale folder for each population
files = [f for f in os.listdir(path+'channel/') if f.endswith(".csv")]
fcs = ['_'.join(x.split('_')[1:-1]) for x in files]
pop = [(x.split('_')[-1]).split('.')[0] for x in files]
data = pd.DataFrame({'file': files, 'fcs': fcs, 'pop': pop, 'plate': [plates.index(p) for p in fcs]}).set_index('file')
dfs_channel = [pd.DataFrame() for i in range(len(plates))]

for f in files:
    fj = pd.read_csv(path+'channel/{}'.format(f))
    fj['Population'] = data.loc[f, 'pop']
    dfs_channel[data.loc[f, 'plate']] = dfs_channel[data.loc[f, 'plate']].append(fj)

dfs_scale = [pd.DataFrame() for i in range(len(plates))]
for f in files:
    fj = pd.read_csv(path+'scale/{}'.format(f))
    fj['Population'] = data.loc[f, 'pop']
    dfs_scale[data.loc[f, 'plate']] = dfs_scale[data.loc[f, 'plate']].append(fj)

# replace the index columns with the non-transformed values from scale
for i in range(len(dfs_channel)):
    dfs_channel[i].loc[:, ['IdxCol', 'IdxRow', 'Time']] = dfs_scale[i].loc[:, ['IdxCol', 'IdxRow', 'Time']]

# transform row index in letter and make Well column. Somehow, the IdxRow index from FJ is reversed
for i in range(len(dfs_channel)):
    dfs_channel[i]["IdxRow"] = dfs_channel[i]["IdxRow"].apply(
    lambda x: [
        "A",
        "B",
        "C",
        "D",
        "E",
        "F",
        "G",
        "H",
        "I",
        "J",
        "K",
        "L",
        "M",
        "N",
        "O",
        "P",
    ][-x]
    )
    dfs_channel[i]["Well"] = dfs_channel[i]["IdxRow"] + dfs_channel[i]["IdxCol"].astype(str)
    dfs_channel[i] = dfs_channel[i].rename(columns={'IdxRow': 'Row', 'IdxCol': 'Column'})

# save one table for each plate
[dfs_channel[i].to_csv(path+'facs_data_P{}.txt'.format(i+1), sep='\t', index=False) for i in range(len(dfs_channel))]

[None, None, None, None, None, None, None, None, None, None, None]